# Rule-based Recommender

This is the most straightforward recommender system. Usually, it is based on rules defined by the business. 

In this case, I will be looking at the following rules:

* Most popular items overall in terms of quantity sold,
* Most popular items by country in terms of quantity sold.

___Items__ are considered to be the unique stock codes._

## Setup

In [15]:
import pandas as pd

from recommender_systems.config import DATA_RAW

In [16]:
data = pd.read_csv(DATA_RAW / "data.csv", encoding="ISO-8859-1")

## Data cleaning

### Investigate data

In [17]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [26]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [27]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Notes:

* The `Quantity` column has negative values. There might be a business reason behind this - for example, back orders, returns, etc. - or it can just be input errors. Since the information on the dataset is limited, I will discard those values.
* The `Description` column has null values which will be discarded. 

### Clean data

In [29]:
data_clean = data[data["Quantity"] > 0].dropna(subset=["Description"])

## Popular items globally

In [ ]:
# Top stock codes by number of transactions
data_clean.groupby(["StockCode", "Description"]).agg({"Quantity": "sum"}).sort_values(
    by="Quantity", ascending=False
).head(10)

,,Quantity
StockCode,Description,
23843,"PAPER CRAFT , LITTLE BIRDIE",80995
23166,MEDIUM CERAMIC TOP STORAGE JAR,78033
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,55047
85099B,JUMBO BAG RED RETROSPOT,48478
85123A,WHITE HANGING HEART T-LIGHT HOLDER,37603
22197,POPCORN HOLDER,36761
84879,ASSORTED COLOUR BIRD ORNAMENT,36461
21212,PACK OF 72 RETROSPOT CAKE CASES,36419
23084,RABBIT NIGHT LIGHT,30788


In [31]:
# Top stock codes by number of transactions
data_clean.groupby(["StockCode", "Description"])["InvoiceNo"].nunique()
# .sort_values(
#     by="Quantity", ascending=False
# ).head(10)

StockCode     Description                        
10002         INFLATABLE POLITICAL GLOBE             71
10080         GROOVY CACTUS INFLATABLE               22
              check                                   1
10120         DOGGY RUBBER                           29
10123C        HEARTS WRAPPING TAPE                    3
                                                     ..
gift_0001_20  to push order througha s stock was      1
gift_0001_30  Dotcomgiftshop Gift Voucher £30.00      7
gift_0001_40  Dotcomgiftshop Gift Voucher £40.00      3
gift_0001_50  Dotcomgiftshop Gift Voucher £50.00      4
m             Manual                                  1
Name: InvoiceNo, Length: 4319, dtype: int64